In [ ]:
#### DESIGN RUN NAME #####################################################################################################
DESIGN_FOLDER     = 'Design_AL'
FOLDER_MATCH      = 'MATCH' #Not needed for now -> This is just for match
FOLDER_PARENT     = '../../design/Parent_1ohp' # -> Adjust, contains ONLY parent pdbs
##########################################################################################################################
#### Input parameters ####################################################################################################
MAX_JOBS          = 12
N_PARENT_JOBS     = 5
MAX_DESIGNS       = 50 #00
KBT_BOLTZMANN     = 0.02
CST_WEIGHT        = 1.0
HIGHSCORE         = 0.93
NEG_BEST          = 0.79
ProteinMPNN_PROB  = 0.0
WT                = "1ohp"
LIGAND            = "5TS"
REPACK            = "36,45,51,67"
MATCH             = "14,18,26,30,55,65,80,82,99,101,112"
DESIGN            = "7,10,11,14,15,18,26,29,30,38,54,55,58,59,63,65,71,73,78"
DESIGN           += ",80,82,84,86,93,95,97,99,101,103,109,112,114,116,121"
ProteinMPNN_T     = "0.1"
SUBMIT_PREFIX     = "AIz" # Adjust, use a short prefix to allow 
BLUEPEBBLE        = True
GRID              = False
BACKGROUND_JOB    = False
ABBIE_LOCAL       = False

if GRID:            ROSETTA_PATH = "/home/bunzelh/rosetta_src_2021.16.61629_bundle/main/source/"      #GRID
if BLUEPEBBLE:      
                    ROSETTA_PATH = "/sw/apps/rosetta/rosetta_src_2021.16.61629_bundle/main/source/"   #BLUEPEBBLE 
                    #BLUEPEBBLE_ACCOUNT = "ptch000361"
                    BLUEPEBBLE_ACCOUNT = "ptch000721"
if BACKGROUND_JOB:  ROSETTA_PATH = "/sw/apps/rosetta/rosetta_src_2021.16.61629_bundle/main/source/"   #BLUEPEBBLE 

if ABBIE_LOCAL:     ROSETTA_PATH = "/home/abbie/rosetta.binary.ubuntu.release-371/main/source"
    
NORM              = {'interface_score': [10, 30], 'total_score': [340, 410], 'catalytic_score': [-40, 0]}
##########################################################################################################################
AIZYME_TOOLS      = "../src/AIzymes_010.ipynb"
%run "$AIZYME_TOOLS"

In [ ]:
%run "$AIZYME_TOOLS"
controller(RESET=False, EXPLORE=True, UNBLOCK_ALL=False, PRINT_VAR=True, PLOT_DATA=False) 

In [ ]:
import pandas as pd

PDB_input  = "/home/abbie/AIzymes/design/Design_AL/../../design/Parent_1ohp/1ohp.pdb"

index = 0

all_scores_df = pd.read_csv("Design_AL/all_scores.csv")

save_cat_res_into_all_scores_df(all_scores_df, index, PDB_input, from_parent_struct=True)

In [ ]:
# WT = "1ohp"
# DESIGN_FOLDER = "Design_AL"
# FOLDER_HOME = f'{os.getcwd()}/{DESIGN_FOLDER}'
index = 0

import os
import pickle

pdb_path = f"{FOLDER_HOME}/{int(index)}/{WT}_Rosetta_Design_{int(index)}.pdb"

rel_pdb_path = os.path.relpath(pdb_path)
print(rel_pdb_path)

out_path = f"{os.path.dirname(rel_pdb_path)}/{WT}_fields.pkl"

In [ ]:
structure_filename = os.path.splitext(pdb_path)[0]

In [ ]:
import subprocess

with open(f"{output_filename}_clean.pdb", "w") as f:
    subprocess.call(['sed', '-n', '/CONECT/q;p' ,pdb_path], stdout=f)

In [ ]:
output = run_command(['sed', '-n', '/CONECT/q;p' ,pdb_path])

with open(f"{output_filename}_clean.pdb", "w") as f:
    f.write(output)

In [ ]:

os.system(f"sed -n '/CONECT/q;p' {pdb_path} > {output_filename}_clean.pdb")
os.system(f"pdb_delelem -H {output_filename}_clean.pdb > {output_filename}_noHclean.pdb")

with open("tleap.in", "w") as f:
        f.write(f"""source leaprc.protein.ff19SB 
source leaprc.gaff
loadamberprep   Input/5TS.prepi
loadamberparams Input/5TS.frcmod
mol = loadpdb {output_filename}_noHclean.pdb
saveamberparm mol {output_filename}.parm7 {output_filename}.rst7
quit
""")
    
os.system("tleap -s -f tleap.in > tleap.tmp")

In [ ]:
run_command(["tleap", "-s", "-f", "tleap.in"])

In [ ]:
os.system(f"ambpdb  -p {output_filename}.parm7 -c {output_filename}.rst7 > {output_filename}_new.pdb")

In [ ]:
f"{os.path.relpath(output_filename)}.parm7"

In [ ]:
pdb_path = f"{FOLDER_HOME}/1/{WT}_Rosetta_Design_1.pdb"

structure_filename = os.path.splitext(pdb_path)[0]

run_command(["python", "FieldTools.py", 
            "-nc", f"{os.path.relpath(structure_filename)}.rst7", 
            "-parm", f"{os.path.relpath(structure_filename)}.parm7", 
            "-out", f"{os.path.relpath(structure_filename)}_fields.pkl", 
            "-target", "Input/field_target.dat", 
            "-solvent", "WAT"])

# subprocess.run(["python", "FieldTools.py", 
#                 "-nc", f"{os.path.relpath(output_filename)}.rst7", 
#                 "-parm", f"{os.path.relpath(output_filename)}.parm7", 
#                 "-out", out_path, 
#                 "-target", "Input/field_target.dat", 
#                 "-solvent", "WAT"])

In [ ]:
python_main(["FieldTools.py", 
                "-nc", f"{os.path.relpath(output_filename)}.rst7", 
                "-parm", f"{os.path.relpath(output_filename)}.parm7", 
                "-out", out_path, 
                "-target", "Input/field_target.dat", 
                "-solvent", "WAT"])

In [ ]:
python_main(["FieldTools.py", "-nc", f"{os.path.relpath(output_filename)}.rst7", "-parm", f"{os.path.relpath(output_filename)}.parm7", "-out", out_path, "-target", "Input/field_target.dat", "-solvent", "WAT"])

In [ ]:
with open(f"{FOLDER_HOME}/3/{WT}_Rosetta_Design_3_fields.pkl", "rb") as f:
    FIELDS_3 = pkl.load(f)

# with open(f"{FOLDER_HOME}/1/{WT}_Rosetta_Design_1_fields.pkl", "rb") as f:
#     FIELDS_1 = pkl.load(f)

# print(FIELDS)

# bond_field = FIELDS[':5TS@C9_:5TS@H04']['Total']

all_fields_3 = FIELDS_3[':5TS@C9_:5TS@H04']
# all_fields_1 = FIELDS_1[':5TS@C9_:5TS@H04']

# print(bond_field[0])
#return (np.mean(bond_fields), list(bond_fields))

In [ ]:
[field[0] for field in all_fields_3.values()]

In [ ]:
print(ALL_SCORES_CSV)

FOLDER_HOME

In [ ]:
def update_efieldsdf(index, index_efields_dict):
    no_residues = len(index_efields_dict)-4

    gen_headers = ["Total","Protein","Solvent","WAT"]
    resi_headers = [f"RESI_{idx}" for idx in range(1,no_residues+1)]
    headers = gen_headers + resi_headers

    fields_list = [field[0] for field in index_efields_dict.values()]

    if not os.path.isfile(f"{FOLDER_HOME}/electric_fields.csv"):       
        fields_df = pd.DataFrame([fields_list], columns=headers, index=[index])
        fields_df.to_csv(f"{FOLDER_HOME}/electric_fields.csv") 

    else:
        fields_df = pd.read_csv(f"{FOLDER_HOME}/electric_fields.csv", index_col=0)
        new_row_df = pd.DataFrame([fields_list], columns=headers, index=[index])
        fields_df = pd.concat([fields_df, new_row_df])
        fields_df.to_csv(f"{FOLDER_HOME}/electric_fields.csv") 
        print(fields_df)

In [ ]:
update_efieldsdf(3, all_fields_3)

#print(len(all_fields_1)-4)

In [ ]:
gen_headers = ["Total","Protein","Solvent","WAT"]
#resi_headers = [f"RESN_{int(idx/2)}" if idx%2 == 0 else f"Efield_{int((idx-1)/2)}" for idx in range(2,252)]
resi_headers = [f"RESI_{idx}" for idx in range(1,127)]

print(resi_headers)

headers = gen_headers + resi_headers

fields_list = [field[0] for field in all_fields_0.values()]

fields_df = pd.DataFrame([fields_list], columns=headers)

fields_1_list = [field[0] for field in all_fields_1.values()]

fields_1_df = pd.DataFrame([fields_1_list], columns=headers, index=[1])

# fields_0_df = pd.DataFrame(all_fields_0, columns = headers)

# fields_0_df

# fields_1_df = pd.DataFrame(all_fields_1, index=[1])

# print(len(fields_0_df.columns))

all = pd.concat([fields_df, fields_1_df])

all

# for column in all.columns:
#     print(column)

In [ ]:
!squeue --me
#!scancel --me

In [ ]:
%run "$AIZYME_TOOLS"
#get_best_structures()